# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686500


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:38,  3.39s/it]

Rendering models:  10%|▉         | 3/31 [00:06<01:08,  2.43s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:47,  1.75s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:31,  1.26s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  6.29it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  6.94it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  6.04it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  7.20it/s]

not-logged-in-20a275b53abec8db9f41    0.019196
not-logged-in-20a275b53abec8db9f41    0.055360
ElisabethB                            0.000087
not-logged-in-31e7b02d91e286c7f9b3    0.001241
shocko61                              0.000094
shocko61                              0.000100
thedarktree                           0.000297
kcarte13                              0.000067
mtajnai                               0.000076
jmfranci                              0.000495
ncaro                                 0.000337
not-logged-in-3bf20897ad3067a962a6    0.000103
not-logged-in-2aaf71de46fc8e66d4f8    0.000334
clairedeu                             0.000081
acapirala                             0.000084
Lavadude                              0.006431
Rollingr                              0.001481
not-logged-in-92531cfb2471bb613fa9    0.004102
Fortnitesucks                         0.000110
nicholas.                             0.008868
not-logged-in-f2d27cddccc2f48b972e    0.000680
vceccare     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())